In [ ]:
import numpy as np
import os
import sys
import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation
from keras.layers import LSTM, Input, Flatten, Concatenate, Embedding, Convolution1D,Dropout, Conv3D, Conv2D, Conv1D, Bidirectional, Reshape, MaxPooling1D, AveragePooling1D
from keras.layers.wrappers import TimeDistributed

from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.normalization import BatchNormalization
from sklearn.preprocessing import label_binarize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.layers.convolutional import Conv3D, ZeroPadding3D
from keras.layers.convolutional import MaxPooling3D, AveragePooling3D
from keras.layers import add
from keras.layers.normalization import BatchNormalization
from keras.utils import plot_model

import argparse

import tensorflow as tf
from keras import backend as K
from keras import regularizers, constraints, initializers, activations
from keras.layers.recurrent import Recurrent
from keras.engine import InputSpec
from keras.callbacks import EarlyStopping,TensorBoard, ModelCheckpoint
from keras_self_attention import SeqSelfAttention
from keras_multi_head import MultiHeadAttention

import pickle as plk

gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
keras.backend.tensorflow_backend.set_session(sess)

In [ ]:
hidden_unit = 512
dropout_rate = 0.35
lstm_cells = 128
classes = 6
batch = 16
epochs = 5000

In [ ]:
data_facial_tra = plk.load(open('../data_clean/split/data_visual_tra', 'rb'))
data_facial_pre_tra = plk.load(open('../data_clean/split/data_visual_pre_tra', 'rb'))

data_facial_tes = plk.load(open('../data_clean/split/data_visual_tes', 'rb'))
data_facial_pre_tes = plk.load(open('../data_clean/split/data_visual_pre_tes', 'rb'))

data_emos_tra = plk.load(open('../data_clean/split/data_emos_tra_', 'rb'))

data_emos_tes = plk.load(open('../data_clean/split/data_emos_tes', 'rb'))

In [ ]:
data = []
loop_n = {0:3,1:2,2:1,3:2,4:2,5:1}
for (faces, faces_pre, emo) in zip(data_facial_tra, data_facial_pre_tra, data_emos_tra):
    for _ in range(loop_n[emo]):
        data.append([faces, faces_pre, emo])
        
np.random.shuffle(data)
data_facial_tra, data_facial_pre_tra, data_emos_tra = [], [], []
for (faces, faces_pre, emo) in data:
    data_facial_tra.append(faces)
    data_facial_pre_tra.append(faces_pre)
    data_emos_tra.append(emo)

In [ ]:
data_type = 'float32'
[data_facial_tra, data_facial_pre_tra, 
 data_facial_tes, data_facial_pre_tes, 
 data_emos_tra, data_emos_tes
]   =    [np.reshape(np.asarray(data_facial_tra, data_type), (9348, 512, 1)), np.reshape(np.asarray(data_facial_pre_tra, data_type), (9348, 512, 1)), 
                                      np.reshape(data_facial_tes, (1623, 512, 1)), np.reshape(data_facial_pre_tes, (1623, 512, 1)),
                                      to_categorical(data_emos_tra, 6), to_categorical(data_emos_tes, 6)
                                      ]

print('data_facial_tra:', np.shape(data_facial_tra))
print('data_facial_tes:', np.shape(data_facial_tes))

print('data_facial_pre_tra:', np.shape(data_facial_pre_tra))
print('data_facial_pre_tes:', np.shape(data_facial_pre_tes))

print('data_emos_tra:', np.shape(data_emos_tra))
print('data_emos_tes:', np.shape(data_emos_tes))

In [ ]:
pre_faces = Input((512,1))
faces = Input((512,1))

Facial_processing = Sequential()
Facial_processing.add(Conv1D(64, 3, strides=1))
Facial_processing.add(EmoEncDec(lstm_cells,lstm_cells, name='EmoEncDec'))
# Facial_processing.add(LSTM(lstm_cells, return_sequences=True, name='EmoEncDec_LSTM'))
Facial_processing.add(MultiHeadAttention(head_num=8))
# Facial_processing.add(Dropout(dropout_rate))
# Facial_processing.add(Flatten())
Facial_processing.add(Bidirectional(LSTM(lstm_cells, recurrent_dropout = 0.2)))


pre_mocab_feature = Facial_processing(pre_faces)
mocab_feature = Facial_processing(faces)


merge = Concatenate(axis=-1)([pre_faces, faces])

R = Flatten()(merge)
R = Dense(64)(R)
emo = Dense(classes, name='emo', activation='softmax')(R)


model = Model(inputs=[pre_faces, faces],outputs=emo)
Facial_processing.summary()
model.summary()

In [ ]:
model.compile(optimizer='adam', 
              loss={'emo':'categorical_crossentropy',
#                         'gen':'categorical_crossentropy',
                    },
              loss_weights={'emo':1.,
#                             'gen':1.,
                            },
              metrics=['acc'])
file_path_root = './facial/'
model_file = file_path_root+'facial_model.h5'
callback_list = [
                    TensorBoard(log_dir=file_path_root),
                    EarlyStopping(
                        monitor='val_acc',
                        patience=500,
                        verbose=1,
                        mode='auto'
                    ),
                    ModelCheckpoint(
                        filepath=model_file,
                        monitor='val_acc',
                        save_best_only='True',
                        verbose=1,
                        mode='auto',
                        period=1
                    )
                    ]

In [ ]:
training = model.fit([data_facial_pre_tra, data_facial_tra], 
          data_emos_tra,
          batch_size=batch,
          epochs=epochs,
          callbacks=callback_list,      
          validation_data=([data_facial_pre_tes, data_facial_tes], 
                           data_emos_tes))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd


model = load_model(model_file, custom_objects={'MultiHeadAttention':MultiHeadAttention, 'EmoEncDec': EmoEncDec})

predicted_test_labels = model.predict([data_facial_pre_tes, data_facial_tes]).argmax(axis=1)
numeric_test_labels = np.array(data_emos_tes).argmax(axis=1)

report_filename = file_path_root+'Results.txt' 

with open(report_filename, 'w', encoding='utf-8') as f:
    print(classification_report(numeric_test_labels, predicted_test_labels, target_names = ['hap', 'sad', 'neu', 'ang', 'exc', 'fru'], digits=4), file=f)
print(classification_report(numeric_test_labels, predicted_test_labels, target_names = ['hap', 'sad', 'neu', 'ang', 'exc', 'fru'], digits=4))
labels = ['hap', 'sad', 'neu', 'ang', 'exc', 'fru']
print('   '+' '.join(labels))
cm = confusion_matrix(y_true=numeric_test_labels.tolist(), y_pred=predicted_test_labels.tolist())
print(cm)


nor_cm = []
for i in range(6):
    row_sum = cm[i].sum()
#     print(row_sum)
    l_n = []
    for j in range(6):
        l_n.append(cm[i][j]/row_sum)
    nor_cm.append(l_n)
    
df_cm = pd.DataFrame(nor_cm, index = [i for i in ['hap', 'sad', 'neu', 'ang', 'exc', 'fru']],
                  columns = [i for i in ['hap', 'sad', 'neu', 'ang', 'exc', 'fru']])

sn.heatmap(df_cm,  annot=True)
plt.savefig(file_path_root+'cm.jpg')